# Alur Preprocessing 2 (Text feature Extraction - Stemming)

Alur dalam preprocessing ini yaitu dengan mengekstraksi kata-kata dengan beberapa metode. Alurnya adalah sebagai berikut

1. Load Data
2. Stemming and Stopwords Removal
4. Vectorize
4. Analyze

## 0. Load Library

In [3]:
import cPickle as pickle
import re #Regex
%matplotlib inline

## 1. Load Data

### Take the Labeled data from Description Section (1.4)

In [4]:
#just load it from the pickle dump
DATA_JUDUL = pickle.load( open( "v1.4\DATA_JUDUL.p", "rb" ) )
DATA_SHORT = pickle.load( open( "v1.4\DATA_SHORT.p", "rb" ) )
DATA_LONG = pickle.load( open( "v1.4\DATA_LONG.p", "rb" ) )

DATA_TARGET = pickle.load( open( "v1.4\DATA_TARGET.p", "rb" ) )

DATA_JUDULStriped = pickle.load( open( "v1.4\DATA_JUDULStriped.p", "rb" ) )
DATA_SHORTStriped = pickle.load( open( "v1.4\DATA_SHORTStriped.p", "rb" ) )
DATA_LONGStriped = pickle.load( open( "v1.4\DATA_LONGStriped.p", "rb" ) )

In [5]:
DATA_JUDULStriped[1]

'dua bumn kurang modal, benih subsidi langka'


## 2. Stemming and Stopwords Removal
### This preproc focus on stemming (menjadikan kata dasar) and Stopwords (Seperti 'dan', 'ada', 'adalah') removal. This Section is for Preproc using Native Package (Sastrawi)


In [6]:
# import StemmerFactory class, github : https://github.com/masdevid/PySastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def rm_stem_native(teks):
    # stemming process
    teks   = stemmer.stem(teks) #Stemming
    return teks

In [7]:
teks_std = DATA_SHORT[98]
teks = DATA_SHORTStriped[98]

print 'Banyaknya kata'
print ('When the data Splitted           : %s' % len(teks_std.split()))
print ('After cleaning Html&Non-Alphanum : %s' % len(teks.split()))
print ('Sastrawi Result count            : %s\n' % len(rm_stem_native(teks).split()))

print teks_std+'\n'
print teks + '\n'
print rm_stem_native(teks) + '\n'

Banyaknya kata
When the data Splitted           : 12
After cleaning Html&Non-Alphanum : 7
Sastrawi Result count            : 7

<img src="http://cdn-media.viva.id/thumbs2/2015/12/18/354946_logo-bank-indonesia_663_382.jpg" align="left" hspace="7" width="100" />Permintaan kredit di enam sektor ekonomi melambat.

permintaan kredit di enam sektor ekonomi melambat.

minta kredit di enam sektor ekonomi lambat



##### Didasarkan pada hasil diatas, proses Preprocessing selanjutnya akan menggunakan package native stemming

In [8]:
#Subsetting the Data, it's going to take more than 30 mins (in my case)
%time DATA_JUDULStem = DATA_JUDULStriped.map(lambda x: rm_stem_native(x))
%time DATA_SHORTStem = DATA_SHORTStriped.map(lambda x: rm_stem_native(x))
%time DATA_LONGStem  = DATA_LONGStriped.map(lambda x: rm_stem_native(x))

Wall time: 2min 29s
Wall time: 1min 50s
Wall time: 28min 11s


In [9]:
import pandas as pd
#Stopwords data source : https://www.illc.uva.nl/Research/Publications/Reports/MoL-2003-02.text.pdf
stopword_html = open("id.stopwords.01.01.2016.txt",'r').read()
stopwords     = stopword_html.split("\n")

from sklearn.feature_extraction.text import CountVectorizer

def tableTop10(a,b,c):    
    def topKWords(docs,k):
        dummy  = docs.tolist()
        countv = CountVectorizer(stop_words=stopwords,decode_error='ignore')
        dummy  = countv.fit_transform(dummy)
        freqs = [(word, dummy.getcol(idx).sum()) for word, idx in countv.vocabulary_.items()]
        #sort from largest to smallest
        return [name for name,freq in sorted(freqs, key = lambda x: -x[1])[0:k]]
    
    top_word = pd.DataFrame([topKWords(a,10),
                             topKWords(b,10),
                             topKWords(c,10)]).transpose()
    return top_word

In [10]:
top_wordBerita1 = tableTop10(DATA_JUDULStriped,DATA_SHORTStriped,DATA_LONGStriped)
top_wordBerita1.columns = ['JudulS','ShortS','LongS']
top_wordBerita2 = tableTop10(DATA_JUDULStem,DATA_SHORTStem,DATA_LONGStem)
top_wordBerita2.columns = ['JudulP','ShortP','LongP']

top_wordBerita = [top_wordBerita1,top_wordBerita2]
top_wordBerita = pd.concat(top_wordBerita,axis=1)
print '10 Kata paling tinggi Frekuensinya Pada teks Sebelum preprocessing'
top_wordBerita

10 Kata paling tinggi Frekuensinya Pada teks Sebelum preprocessing


,JudulS,ShortS,LongS,JudulP,ShortP,LongP
0,saham,pt,persen,saham,pt,persen
1,com,tbk,2016,com,tbk,usaha
2,jpnn,indonesia,indonesia,jpnn,indonesia,2016
3,reklamasi,persen,jakarta,harga,laku,indonesia
4,triliun,astra,pt,reklamasi,persen,laku
5,harga,2016,harga,triliun,usaha,jakarta
6,astra,harga,negara,turun,harga,harga
7,indonesia,jakarta,saham,usaha,astra,pt
8,pemerintah,saham,perusahaan,jual,turun,kerja
9,turun,triliun,triliun,indonesia,2016,turun


In [11]:
#Saving the data to file , it's 30mins processing time dude!
pickle.dump( DATA_JUDULStem, open( "v1.5\DATA_JUDULStem.p", "wb" ) )
pickle.dump( DATA_SHORTStem, open( "v1.5\DATA_SHORTStem.p", "wb" ) )
pickle.dump( DATA_LONGStem, open( "v1.5\DATA_LONGStem.p", "wb" ) )
pickle.dump( DATA_TARGET, open( "v1.5\DATA_TARGET.p", "wb" ) )